In [1]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

from richness_mass_calib import create_richness_mass_calib

In [2]:
# #Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [3]:
# Data
DC2_halos_m200c = fits.open(
    "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
)
dt_halos = Table(DC2_halos_m200c[1].data)
# dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']

In [4]:
rmdata = create_richness_mass_calib(dt_halos)
lnrich_ext = Nc.ClusterMassLnrichExt()
lnrich_ext.param_set_by_name("cut", 1e15)
lnrich_ext.param_set_by_name("cutM1", 0)
lnrich_ext.param_set_by_name("cutZ1", 0)


mset = Ncm.MSet()
mset.set(lnrich_ext)

rmdata.m2lnL_val(mset)

20078883.523066062

In [5]:
dset = Ncm.Dataset.new()
dset.append_data(rmdata)
lh = Ncm.Likelihood.new(dset)

mset.param_set_all_ftype(Ncm.ParamType.FREE)
mset.param_set_ftype(7000, 12, Ncm.ParamType.FIXED)
mset.param_set_ftype(7000, 13, Ncm.ParamType.FIXED)
mset.param_set_ftype(7000, 14, Ncm.ParamType.FIXED)
mset.prepare_fparam_map()

fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL)
fit.log_info()
fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)


#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.19                [FREE]
#   -    muM1[01]:  0.868588963806504   [FREE]
#   -    muZ1[02]: -0.304006137332276   [FREE]
#   -    muM2[03]:  0                   [FREE]
#   -    muZ2[04]:  0                   [FREE]
#   -    muMZ[05]:  0                   [FREE]
#   -  sigma0[06]:  0.33                [FREE]
#   - sigmaM1[07]: -0.0347435585522601  [FREE]
#   - sigmaZ1[08]:  0                   [FREE]
#   - sigmaM2[09]:  0                   [FREE]
#   - sigmaZ2[10]:  0                   [FREE]
#   - sigmaMZ[11]:  0                   [FREE]
#   -     cut[12]:  1e+15               [FIX

In [6]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.95849990149314    [FREE]
#   -    muM1[01]:  0.834834799745832   [FREE]
#   -    muZ1[02]:  0.695004688438083   [FREE]
#   -    muM2[03]:  0.0421606025663114  [FREE]
#   -    muZ2[04]: -0.051715032078255   [FREE]
#   -    muMZ[05]:  0.0402769634041103  [FREE]
#   -  sigma0[06]:  0.306313313048325   [FREE]
#   - sigmaM1[07]:  0.00737198200713169 [FREE]
#   - sigmaZ1[08]: -0.0212988049057424  [FREE]
#   - sigmaM2[09]:  0.015610993686449   [FREE]
#   - sigmaZ2[10]:  0.00480098229907118 [FREE]
#   - sigmaMZ[11]:  0.0111750633563743  [FREE]
#   -     cut[12]:  1e+15               [FIX

In [ ]:
nwalkers = 500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("model_extz2.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.95849990149314    [FREE]
#   -    muM1[01

In [ ]:
esmcmc.mean_covar()
fit.log_covar()

In [ ]:
fit.log_covar()

In [ ]:
fit.log_info()